In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from IPython.display import Image
from datetime import datetime

In [3]:
# usecols 뽑을 데이터 지정하는 코드, converters 종목코드 str으로 바꿔주는 코드

### 종목코드 앞자리가 0이므로 스트링으로 지정해주지않으면 0이 사라짐 ###

# 상장회사 목록
df2 = pd.read_csv('Dart_codelist_last30_2.csv', usecols = ['종목명','종목코드','기업코드','수정일'], converters={'종목코드':str,'기업코드':str})
df2

,종목명,종목코드,기업코드,수정일
0,지놈앤컴퍼니,314130,01276026,20210216
1,웹젠,069080,00405320,20210215
2,우리넷,115440,00664048,20210217
3,마이크로디지탈,305090,01267967,20210217
4,나노,187790,00994994,20210216
5,손오공,066910,00306870,20210216
6,새로닉스,042600,00147994,20210217
7,피씨디렉트,051380,00297934,20210217
8,이큐셀,160600,00818472,20210125
9,케이엠제약,225430,01095704,20210126


In [4]:
## 종목코드의 기업명 불러낼 dict 만들기
## ∇여기부터

corp_dict = {}

for i in range(len(df2['종목코드'])):  # 아래 크롤링에 포함된 함수에 기업명 컬럼을 추가하기 위해 dict 형태로 전처리
    corp_dict[df2.loc[i]['종목코드']] = str(df2.loc[i]['종목명'])
corp_dict

{'314130': '지놈앤컴퍼니',
 '069080': '웹젠',
 '115440': '우리넷',
 '305090': '마이크로디지탈',
 '187790': '나노',
 '066910': '손오공',
 '042600': '새로닉스',
 '051380': '피씨디렉트',
 '160600': '이큐셀',
 '225430': '케이엠제약',
 '331520': '교보9호스팩',
 '086220': '네추럴FNP',
 '215100': '로보로보',
 '069730': 'DSR제강',
 '355150': '교보10호기업인수목적',
 '339950': '아이비김영',
 '233250': '메디안디노스틱',
 '357250': '미래에셋맵스리츠',
 '101680': '한국정밀기계',
 '318410': '비비씨',
 '122350': '삼기',
 '245450': '씨앤에스링크',
 '033270': '유나이티드',
 '263860': '지니언스',
 '033540': '파라텍',
 '015360': '예스코홀딩스',
 '025320': '시노펙스',
 '012690': '모나리자',
 '004870': '티웨이홀딩스',
 '264660': '씨앤지하이테크',
 '004450': '삼화왕관',
 '035150': '백산',
 '239890': '피엔에이치테크',
 '215050': '비엔디생활건강',
 '003380': '하림지주',
 '096630': '에스코넥',
 '001360': '삼성제약',
 '133820': '화인베스틸',
 '006250': '극동전선',
 '214270': '퓨쳐스트림네트웍스',
 '023460': 'CNH',
 '033020': '세아메탈',
 '052710': '아모텍',
 '065570': '삼영이엔씨',
 '086710': '선진뷰티사이언스',
 '285130': 'SK케미칼',
 '029780': '삼성카드',
 '290510': '코리아센터',
 '084690': '대상홀딩스',
 '032540': 'TJ미디어',
 '196

# ★고쳐야됨 밑에꺼 [:100]

In [5]:
### 
com_code = df2['종목코드'][:100]
com_code

0     314130
1     069080
2     115440
3     305090
4     187790
5     066910
6     042600
7     051380
8     160600
9     225430
10    331520
11    086220
12    215100
13    069730
14    355150
15    339950
16    233250
17    357250
18    101680
19    318410
20    122350
21    245450
22    033270
23    263860
24    033540
25    015360
26    025320
27    012690
28    004870
29    264660
       ...  
70    034830
71    134380
72    044990
73    085660
74    081000
75    363280
76    092870
77    072020
78    244880
79    058480
80    311060
81    318010
82    040610
83    163560
84    115610
85    211050
86    037350
87    222980
88    191410
89    155900
90    143240
91    340350
92    317690
93    293780
94    348350
95    067160
96    323350
97    036710
98    222800
99    304100
Name: 종목코드, Length: 100, dtype: object

In [6]:
def reverse_rolling(series, windows_):

    # 시리즈 인덱스 저장하기
    index = series.index
    # 컬럼 뒤집기
    series = pd.DataFrame(series.iloc[::-1])
    # 컬럼뒤집어서 이동평균
    series = series.rolling(window = int(windows_)).mean() 
    # 컬럼 다시 뒤집기
    series = series.iloc[::-1]
    # 저장했던 index 다시 집어넣기
    series['index'] = index
    # index 재설정하기
    series = series.set_index('index')


    return series

In [7]:
# 종목코드 기준으로 df별로 돌릴거

def cross_score(df):

    df['ma5']=reverse_rolling(df['주식_종가'],5).fillna(value=0)
    df['ma5_temp'] = df['ma5'].shift(fill_value=0).fillna(value=0)
    df['ma5_slope'] = pd.to_numeric(((df['ma5']-df['ma5_temp'])/(df['ma5_temp']))\
            .replace(np.inf,"0").fillna(value=0)).shift(periods=-1,fill_value=0)


# pd.to_numeric(df['ma5_slope']) str->int

## 20일 기준 이동평균
    df['ma20']=reverse_rolling(df['주식_종가'],20).fillna(value=0)
    df['ma20_temp'] = df['ma20'].shift(fill_value=0).fillna(value=0)
    df['ma20_slope']=pd.to_numeric(((df['ma20']-df['ma20_temp'])/(df['ma20_temp']))\
            .replace(np.inf,"0").fillna(value=0)).shift(periods=-1,fill_value=0)

## 60일 기준 이동평균
    df['ma60']=reverse_rolling(df['주식_종가'],60).fillna(value=0)
    df['ma60_temp'] = df['ma60'].shift(fill_value=0).fillna(value=0)
    df['ma60_slope'] = pd.to_numeric(((df['ma60']-df['ma60_temp'])/(df['ma60_temp']))\
            .replace(np.inf,"0").fillna(value=0)).shift(periods=-1,fill_value=0)

## 120일 기준 이동평균
    df['ma120']=reverse_rolling(df['주식_종가'],120).fillna(value=0)
    df['ma120_temp'] = df['ma120'].shift(fill_value=0).fillna(value=0)
    df['ma120_slope']=pd.to_numeric(((df['ma120']-df['ma120_temp'])/(df['ma120_temp']))\
            .replace(np.inf,"0").fillna(value=0)).shift(periods=-1,fill_value=0)
    
## 이동평균선 기준 scoring
## 향후 과제로 해당 weight를 찾는걸 해도 될듯.
    df['weighted_avg']= 0.65 * df['ma5_slope']\
                        + 0.2 * df['ma20_slope'] \
                        + 0.1 * df['ma60_slope'] \
                        + 0.05 * df['ma120_slope']

    del(df['ma20_temp'])
    del(df['ma5_temp'])
    del(df['ma60_temp'])
    del(df['ma120_temp'])
    
    

    return df

# ★고쳐야됨 for i in range(3)

In [8]:
#############최종본
########빈리스트 만들기##########
df_list = []

###############################

df_total = pd.DataFrame()
temp_df = pd.DataFrame()


###################
# step1. 원하는 시세 페이지 범위 정하기 

page_number_lst = []  
for i in range(3):     # 원하는 페이지수만큼 range의 파라미터 설정 
    page_number_lst.append(i+1)

# step2. 네이버 증권의 시세 페이지에서 종가 및 날짜 데이터 크롤링해서, 기업별 종목코드로 DataFrmae 만들기     

for code in com_code:
    df = pd.DataFrame()   # 빈 DataFrame 만들기 (종목별로 DataFrmae 값을 넣기 위한 틀)
    list_date = []        # 리스트 초기화 
    list_endprice = []
    corp_total = []
    corp_name = []
    for _page in page_number_lst: # 1페이지부터 n페이지까지 크롤링  (Step1 참조.)

        url_head = 'https://finance.naver.com/item/sise_day.nhn?code={}&page={}'.format(code,_page)  # 종목코드와 페이지를 넣는 url 생성
        headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36'}
        web = requests.get(url_head, headers = headers).content #자료 요청
        soup = BeautifulSoup( web, 'html.parser') # html 파싱

        for tr in soup.find('table').find_all('tr'): # 일별시세 웹에서 ['날짜'.'종가','전일비'.. ] 행 크롤링, (table 태그안에 tr태그의 텍스트) 이미지참조.
            try:
                if len(tr.find_all("td")[0].get_text()) != 10: # 날짜가 비워진 셀은 패스 / 
                    pass
                else:
                    if tr.find_all("td")[0].get_text() in list_date and int(tr.find_all("td")[0].get_text()[:4]) < 2021: # 동일한 날짜가 기존에 있었는지 체크 (중복 페이지 예외처리)
                        break
                    else:
                        list_date.append(tr.find_all("td")[0].get_text())   # 날짜 데이터 삽입
                        list_endprice.append(tr.find_all('td')[1].get_text())  # 종가 데이터 삽입
                        corp_total.append(code)  # 종목 코드 삽입
                        corp_name.append(corp_dict[code]) # 기업명 삽입
            except:
                pass
    for i in range(len(list_endprice)):
        list_endprice[i] = list_endprice[i].replace(",","")   # 추후 계산을 위해 값의 쉼표(,) 제거
        
#     list_endprice = list(map(int,list_endprice))  #실행시 오류가 나서 제외

    df['날짜'] = list_date               # 날짜 컬럼에 날짜 Valuse 삽입
    df['주식_종가'] = list_endprice      # 주식종가 컬럼에 종가 Valuse 삽입
    df['종목코드'] = corp_total          # 종목코드 컬럼에 코드 Valuse 삽입
    df['종목명'] = corp_name             # 종목명 컬럼에 기업명 Valuse 삽입
    
    globals()['df_{}'.format(code)]= cross_score(df[df['종목코드']==code])     # 변수를 전역변수로 변환해서, 추후 함수 파라미터(변수)로 활용
    temp_df = globals()['df_{}'.format(code)]
    df_total = pd.concat([df_total,temp_df],ignore_index=True, sort=True) # 하나의 Data Frame으로 합치기
    df_list.append(['df_{}'.format(code)])   # 만들어진 기업별 df명 리스트화 (확인용)


In [24]:
len(df_list)

100

In [11]:
df_total[df_total['weighted_avg']>0]

,ma120,ma120_slope,ma20,ma20_slope,ma5,ma5_slope,ma60,ma60_slope,weighted_avg,날짜,종목명,종목코드,주식_종가
4,0.0,0.0,35727.5,0.008117,34510.0,0.002028,0.0,0.0,0.002942,2021.06.15,지놈앤컴퍼니,314130,34600
5,0.0,0.0,36017.5,0.005969,34580.0,0.002024,0.0,0.0,0.002510,2021.06.14,지놈앤컴퍼니,314130,34400
6,0.0,0.0,36232.5,0.005451,34650.0,0.004906,0.0,0.0,0.004279,2021.06.11,지놈앤컴퍼니,314130,34850
7,0.0,0.0,36430.0,0.007755,34820.0,0.006031,0.0,0.0,0.005471,2021.06.10,지놈앤컴퍼니,314130,34550
8,0.0,0.0,36712.5,0.010759,35030.0,0.007137,0.0,0.0,0.006791,2021.06.09,지놈앤컴퍼니,314130,34150
9,0.0,0.0,37107.5,0.011386,35280.0,0.007086,0.0,0.0,0.006883,2021.06.08,지놈앤컴퍼니,314130,34950
11,0.0,0.0,0.0,0.000000,36010.0,0.006109,0.0,0.0,0.003971,2021.06.04,지놈앤컴퍼니,314130,35700
12,0.0,0.0,0.0,0.000000,36230.0,0.001656,0.0,0.0,0.001076,2021.06.03,지놈앤컴퍼니,314130,35600
16,0.0,0.0,0.0,0.000000,35510.0,0.001690,0.0,0.0,0.001098,2021.05.28,지놈앤컴퍼니,314130,36800
17,0.0,0.0,0.0,0.000000,35570.0,0.012932,0.0,0.0,0.008406,2021.05.27,지놈앤컴퍼니,314130,35900


# Part3 시작

In [20]:
df_total[df_total['날짜']==today_]

,ma120,ma120_slope,ma20,ma20_slope,ma5,ma5_slope,ma60,ma60_slope,weighted_avg,날짜,종목명,종목코드,주식_종가
0,0.0,0.0,35610.00,-0.002317,36310.0,-0.010190,0.0,0.0,-0.007087,2021.06.21,지놈앤컴퍼니,314130,36250
30,0.0,0.0,32062.50,0.000546,32060.0,0.004055,0.0,0.0,0.002745,2021.06.21,웹젠,069080,31700
60,0.0,0.0,8753.50,-0.002228,8784.0,-0.004326,0.0,0.0,-0.003257,2021.06.21,우리넷,115440,8910
90,0.0,0.0,25430.00,0.001376,24470.0,0.011443,0.0,0.0,0.007713,2021.06.21,마이크로디지탈,305090,23800
120,0.0,0.0,1790.25,-0.005027,1859.0,0.000000,0.0,0.0,-0.001005,2021.06.21,나노,187790,1830
150,0.0,0.0,3075.25,-0.003414,3056.0,0.002291,0.0,0.0,0.000806,2021.06.21,손오공,066910,3010
180,0.0,0.0,10460.50,-0.003585,11050.0,0.000905,0.0,0.0,-0.000129,2021.06.21,새로닉스,042600,11050
210,0.0,0.0,11412.50,-0.002191,11580.0,-0.003454,0.0,0.0,-0.002683,2021.06.21,피씨디렉트,051380,11650
240,0.0,0.0,3100.00,0.000000,3100.0,0.000000,0.0,0.0,0.000000,2021.06.21,이큐셀,160600,3100
270,0.0,0.0,2981.75,-0.004276,3038.0,-0.002304,0.0,0.0,-0.002353,2021.06.21,케이엠제약,225430,3040


In [21]:
# datetime.today()            # 현재 날짜 가져오기
# datetime.today().year        # 현재 연도 가져오기
# datetime.today().month      # 현재 월 가져오기
# datetime.today().day        # 현재 일 가져오기
# datetime.today().hour        # 현재 시간 가져오기


In [22]:
month_ =''
day_ = ''
if len(str(datetime.today().month)) ==1 :
    month_ = str('0'+str(datetime.today().month))
else:
    pass
if len(str(datetime.today().day)) == 1:
    day_ = str('0'+str(datetime.today().day))
else:
    day_=str(datetime.today().day)


In [23]:
type(datetime.today().day)

int

In [18]:
today_ = str(datetime.today().year)+'.'+str(month_)+'.'+str(day_)

In [19]:
today_

'2021.06.21'

In [37]:
today = '20210621'

In [43]:
import datetime

def GetQuaterLastDate(today):
    if today.month < 4:
        targetMonth = 3
        targetDay = 31
    elif today.month < 7:
        targetMonth = 6
        targetDay = 30
    elif today.month < 10:
        targetMonth = 9
        targetDay = 30
    else: targetMonth = 12
        targetDay = 31
        targetDate = datetime.datetime(today.year, targetMonth, targetDay)
    print(targetDate)

    GetQuaterLastDate(2)
    
# if __name__ == "__main__":
#     print(GetQuaterLastDate(datetime.datetime(2016, 2 , 16))) # 2016-03-31 00:00:00 print(GetQuaterLastDate(datetime.datetime(2019, 11, 16))) # 2019-12-31 00:00:00
#     print(GetQuaterLastDate(datetime.datetime(2019, 11, 16))) # 2019-12-31 00:00:00

IndentationError: unexpected indent (<ipython-input-43-144c69eb4b20>, line 14)

In [36]:
type(int('2021.06.21'))

SyntaxError: unexpected EOF while parsing (<ipython-input-36-727cd1ad4635>, line 1)